# Paso 1 - Configuración Inicial (Gemini y Groq)

En esta primera sección se realiza la **configuración de entorno** para trabajar con modelos LLM 
(Gemini y Groq) dentro de LangChain.  

### ¿Qué se hace aquí?
1. **Definición de claves API**: se cargan en variables de entorno para poder autenticar con los servicios.
2. **Definición de USER_AGENT**: necesario para evitar errores con Gemini.
3. **Importación de librerías principales**: carga de documentos, splitters, vectorstores, chains y prompts.
4. **Inicialización de modelos**: 
   - Gemini (`gemini-2.5-flash`) → modelo de Google.
   - Groq (`llama-3.1-8b-instant`) → modelo de Groq.
5. **Prueba de conexión**: se hace una invocación sencilla para confirmar que los modelos están activos.


In [1]:
#CONFIGURACIÓN INICIAL (Gemini y Groq)


import os
from dotenv import load_dotenv

# 1. Cargar variables desde el archivo .env
load_dotenv()

# 2. Leer variables de entorno
gemini_api = os.getenv("GOOGLE_API_KEY")
groq_api   = os.getenv("GROQ_API_KEY")
openai_api = os.getenv("OPENAI_API_KEY")
user_agent = os.getenv("USER_AGENT")

print(f" Gemini Key cargada: {'OK' if gemini_api else 'NO'}")
print(f" Groq Key cargada:   {'OK' if groq_api else 'NO'}")
print(f" OpenAI Key cargada: {'OK' if openai_api else 'NO'}")
print(f" User Agent: {user_agent if user_agent else 'NO definido'}")

# 3. Imports principales de LangChain
# ---- Carga de documentos y preprocesamiento ----
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ---- VectorStores (almacenamiento de embeddings) ----
from langchain_community.vectorstores import Chroma

# ---- Cadenas (chains) para recuperación y combinación ----
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# ---- Prompts, parsers y memoria ----
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.memory import ConversationSummaryBufferMemory

# ---- Herramientas útiles (para Jupyter) ----
import ipywidgets as widgets
from IPython.display import display, clear_output

# 4. Inicialización de Modelos LLM: Groq y Gemini
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

# 5. Verificación de conexión rápida con ambos modelos
try:
    # Cliente Gemini
    gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=gemini_api)
    resp_gemini = gemini_llm.invoke("Hola Gemini, ¿me escuchas?")
    print(" Gemini responde:", resp_gemini.content)

    # Cliente Groq
    groq_llm = ChatGroq(model="llama-3.1-8b-instant", api_key=groq_api)
    resp_groq = groq_llm.invoke("Hola Groq, ¿me escuchas?")
    print(" Groq responde:", resp_groq.content)

    print("🚀 Paso 1 completado correctamente: Conexión establecida con Gemini y Groq.")

except Exception as e:
    print("❌ Error al conectar con los modelos:", e)


 Gemini Key cargada: OK
 Groq Key cargada:   OK
 OpenAI Key cargada: OK
 User Agent: OscarGaitan-LangChainApp/1.0
 Gemini responde: ¡Hola! Sí, te escucho perfectamente. ¿En qué puedo ayudarte?
 Groq responde: Hola, no soy Groq, soy un modelo de lenguaje artificial llamado Llama (Large Language Model Meta AI). Estoy aquí para ayudarte con cualquier pregunta o tema que desees discutir. ¿En qué puedo ayudarte hoy?
🚀 Paso 1 completado correctamente: Conexión establecida con Gemini y Groq.


# Paso 2 - Carga de Documentos

En esta sección se realiza la **carga de los documentos PDF** que serán usados como base de conocimiento en el sistema de preguntas y respuestas.  

### ¿Qué se hace aquí?
1. **Definición de rutas de documentos**: se establece la lista de archivos PDF a utilizar (en este caso 3).  
2. **Uso de `PyPDFLoader`**: se emplea este loader de `langchain_community` para leer el contenido de cada PDF, página por página.  
3. **Manejo de errores**: si algún archivo no existe o está dañado, se captura la excepción y se notifica sin detener el flujo.  
4. **Reporte de carga**: se imprime por consola el nombre del archivo y el número de páginas extraídas.  
5. **Consolidación de documentos**: todos los textos cargados se almacenan en una sola lista `documents`.  



In [2]:
# CARGA DE DOCUMENTOS PDF (limpio)

import os

# Rutas de los PDFs a cargar (ajusta si hace falta)
pdf_paths = [
    "docs/constitucio-politica-colombia-1991.pdf",
    "docs/reglamento_estudiantil.pdf",
    "docs/codigo_sustantivo_trabajo.pdf"
]

# Lista donde se almacenarán todos los documentos (cada elemento es una página)
documents = []

for path in pdf_paths:
    # 1. Validación rápida: el archivo existe en disco?
    if not os.path.exists(path):
        print(f"⚠️ Archivo no encontrado: {path}")
        continue

    try:
        # 2. PyPDFLoader ya fue importado en Paso 1 -> lo usamos directamente
        loader = PyPDFLoader(path)
        docs = loader.load()               # devuelve lista de Document (página por página)
        documents.extend(docs)
        print(f"✅ Cargado: {path} ({len(docs)} páginas)")
    except Exception as e:
        print(f"⚠️ No se pudo cargar {path}: {e}")

# Reporte final
print(f"\n Total de páginas/documentos cargados: {len(documents)}")

# Pequeña verificación visual (primeros 300 caracteres de la primera página)
if documents:
    print("\n--- Ejemplo (primera página, preview) ---")
    print(documents[0].page_content[:300].replace("\n", " ") + "...")


✅ Cargado: docs/constitucio-politica-colombia-1991.pdf (108 páginas)
✅ Cargado: docs/reglamento_estudiantil.pdf (41 páginas)
✅ Cargado: docs/codigo_sustantivo_trabajo.pdf (537 páginas)

 Total de páginas/documentos cargados: 686

--- Ejemplo (primera página, preview) ---
CONSTITUCION POLITICA DE COLOMBIA 1991          PREAMBULO    EL PUEBLO DE COLOMBIA    en ejercicio de su poder soberano, representado por sus delegatarios a la Asamblea  Nacional Constituyente, invocando la protección de Dios, y con el fin de fortalecer la  unidad de la Nación y asegurar a sus integ...


# Paso 3 - Transformación y Split de Textos

En esta sección se realiza la **segmentación del contenido** cargado desde los documentos PDF, con el fin de preparar los textos para su indexación y posterior recuperación eficiente.  

### ¿Qué se hace aquí?
1. **Configuración del `RecursiveCharacterTextSplitter`**:  
   - `chunk_size=1000`: define la longitud máxima de cada fragmento de texto.  
   - `chunk_overlap=200`: establece un solapamiento entre fragmentos, lo cual ayuda a no perder contexto entre chunks consecutivos.  
   - `length_function=len`: especifica la función usada para medir la longitud de los fragmentos.  
   - `is_separator_regex=False`: indica que los separadores definidos no son expresiones regulares.  

2. **División de documentos**: cada `Document` (página de PDF) se corta en fragmentos más pequeños llamados *chunks*.  

3. **Almacenamiento en `splits`**: los fragmentos resultantes se guardan en una lista para ser usados en el paso siguiente (creación de embeddings y vectorstore).  

4. **Reporte de generación**: se imprime por consola la cantidad total de chunks producidos.  

5. **Vista previa**: se muestra un ejemplo del contenido del primer chunk para comprobar que la segmentación se realizó correctamente.


In [8]:
# TRANSFORMACIÓN Y SPLIT DE TEXTOS


# 1. Configuración del splitter
#    - chunk_size: tamaño máximo de cada fragmento de texto.
#    - chunk_overlap: número de caracteres que se solapan entre fragmentos consecutivos.
#    - Esto asegura que no se pierda contexto en los límites de los chunks.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,     
    chunk_overlap=200,   
    length_function=len,
    is_separator_regex=False,
)

# 2. División de los documentos en fragmentos (chunks)
splits = text_splitter.split_documents(documents)

# 3. Reporte de resultados
print(f" Total de chunks generados: {len(splits)}")

# 4. Vista previa de un chunk (si existe al menos uno)
if splits:
    print("\nEjemplo de un chunk:\n")
    print(splits[0].page_content[:500])  # muestra 500 caracteres del primer chunk
else:
    print("⚠️ No se generaron chunks. Verifique la carga de documentos en el paso anterior.")


 Total de chunks generados: 2244

Ejemplo de un chunk:

CONSTITUCION POLITICA DE COLOMBIA 1991 
 
 
 
 
PREAMBULO 
 
EL PUEBLO DE COLOMBIA 
 
en ejercicio de su poder soberano, representado por sus delegatarios a la Asamblea 
Nacional Constituyente, invocando la protección de Dios, y con el fin de fortalecer la 
unidad de la Nación y asegurar a sus integrantes la vida, la convivencia, el trabajo, la 
justicia, la igualdad, el conocimiento, la libertad y la paz, dentro de un marco jurídico, 
democrático y participativo que garantice un orden político,


***Embeddings + Vector DB (Chroma/FAISS)***

# Paso 4A - Indexación y Actualización de la Base Vectorial  

En esta sección se realiza la **creación o actualización** de la base de datos vectorial con los *chunks* generados previamente.  

### ¿Qué se hace aquí?
1. **Inicialización del modelo de embeddings**: se emplea un modelo de *Sentence Transformers* para convertir los fragmentos de texto en representaciones numéricas (**vectores**).  
2. **Definición de persistencia**: se establece una carpeta llamada `chroma_db` donde se almacenará la base vectorial.  
3. **Creación de la base**: si no existe una base previa, se crea desde cero con los *chunks* actuales.  
4. **Actualización incremental**: si ya existe una base, se cargan los datos guardados y se agregan únicamente los nuevos *chunks*.  
5. **Guardado de cambios**: se asegura la persistencia de la base para que pueda reutilizarse en futuras ejecuciones.  
6. **Reporte final**: se imprime la cantidad total de documentos indexados tras la creación o actualización.  


In [4]:
# EMBEDDINGS Y VECTORSTORE (CHROMA) con HuggingFace + Progreso


import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from tqdm import tqdm  # para la barra de progreso

# Usar embeddings locales gratuitos
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Carpeta de persistencia para la base vectorial
persist_directory = "chroma_db"

texts = [doc.page_content for doc in splits]
metadatas = [doc.metadata for doc in splits]

# Si la base vectorial no existe, se crea
if not os.path.exists(persist_directory):
    print(f"Creando base vectorial desde cero con {len(splits)} chunks...")
    vectorstore = Chroma.from_texts(
        texts=texts,
        embedding=embedding_model,
        metadatas=metadatas,
        persist_directory=persist_directory
    )
    vectorstore.persist()
else:
    # Si ya existe, se carga
    print("Cargando base vectorial existente...")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_model
    )

    # Aquí se decides si agregar los nuevos documentos
    if texts:
        print(f" Agregando {len(texts)} nuevos chunks...")
        vectorstore.add_texts(texts=texts, metadatas=metadatas)
        vectorstore.persist()

# Reporte final
print("\n Vectorstore listo con HuggingFace.")
print(" Total de documentos indexados:", vectorstore._collection.count())


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14796\3681884105.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


NameError: name 'splits' is not defined

# Paso 4B - Carga de la Base Vectorial Existente  

En esta sección se realiza la **carga de la base vectorial previamente guardada** para reutilizarla en consultas y búsquedas semánticas.  

### ¿Qué se hace aquí?
1. **Inicialización del modelo de embeddings**: se asegura que se use el mismo modelo con el que fue creada la base.  
2. **Carga desde persistencia**: se abre la base guardada en la carpeta `chroma_db`.  
3. **Confirmación de carga**: se muestra en consola un mensaje confirmando que la base fue cargada con éxito.  
4. **Reporte final**: se imprime el número total de documentos actualmente indexados en la base.  


In [3]:
# Cargar base vectorial existente (solo uso, sin indexar)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

persist_directory = "chroma_db"

vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model
)

print(" Base vectorial lista para consultas.")



C:\Users\USUARIO\AppData\Local\Temp\ipykernel_6540\3676209487.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_6540\3676209487.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


 Base vectorial lista para consultas.


# Paso 5 - Creación del Chain RAG (Retriever → LLM con Groq)

En esta sección se implementa el **pipeline RAG (Retrieval-Augmented Generation)**, que conecta la base vectorial con un modelo de lenguaje para responder preguntas en lenguaje natural basadas en los documentos indexados.  

### ¿Qué se hace aquí?
1. **Carga de la base vectorial (`Chroma`)**:  
   Se abre la base persistida en la carpeta `chroma_db`, asegurando que se utilice el mismo modelo de embeddings configurado en el paso anterior.  

2. **Creación del *retriever***:  
   - El retriever se encarga de buscar los *chunks* más relevantes frente a una consulta del usuario.  
   - Se define el `search_type="similarity"` para recuperar fragmentos con mayor similitud semántica.  
   - Se establece `k=3`, lo que significa que se recuperan los tres fragmentos más relevantes por cada consulta.  

3. **Definición del LLM con Groq**:  
   - Se sustituye el uso de Gemini por **Groq**, aprovechando su rapidez y eficiencia en consultas de recuperación.  
   - Se configura con `temperature=0` para obtener respuestas determinísticas y consistentes.  

4. **Construcción del Chain RAG**:  
   - Se conecta el retriever con el LLM de Groq mediante el `RetrievalQA`.  
   - El chain recibe una pregunta del usuario, busca los *chunks* relevantes y genera una respuesta usando el contexto de los documentos.  

5. **Reporte final**:  
   Se confirma que el **Chain RAG con Groq** está creado y listo para responder preguntas de forma contextualizada, basándose en la información previamente indexada.  


In [4]:
# ============================================
# PASO 5 - CREAR EL CHAIN RAG (Retriever → LLM con Groq)
# ============================================

from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq  # 👈 usamos Groq en lugar de Gemini

# 1. Cargar el vectorstore persistido en "chroma_db"
# -------------------------------------------------
# Aquí reabrimos la base vectorial creada en el paso anterior.
# Es importante que usemos el mismo modelo de embeddings para mantener consistencia.
vectorstore = Chroma(
    persist_directory="chroma_db",
    embedding_function=embedding_model
)

# 2. Crear un retriever (buscador de chunks relevantes)
# ----------------------------------------------------
# El retriever se encarga de buscar los fragmentos de texto más relevantes
# frente a la consulta que realice el usuario.
retriever = vectorstore.as_retriever(
    search_type="similarity",  # también puede ser "mmr" (Maximal Marginal Relevance) para más diversidad
    search_kwargs={"k": 3}     # número de chunks a recuperar (aumentar k = más contexto)
)

# 3. Definir el LLM (Groq)
# ------------------------
# Configuramos el modelo Groq con temperatura=0 para que las respuestas
# sean más determinísticas (menos aleatorias).
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=groq_api,
    temperature=0
)

# 4. Conectar el retriever con el LLM → Chain RAG
# -----------------------------------------------
# El chain conecta la recuperación de chunks con la generación de respuesta
# en lenguaje natural. El flujo es:
#   Pregunta → Retriever (busca contexto) → Groq (genera respuesta)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # "stuff" concatena los chunks antes de pasarlos al LLM
)

print("✅ Chain RAG creado con Groq y listo para responder preguntas.")


✅ Chain RAG creado con Groq y listo para responder preguntas.


# Paso 6 - Mensajes y Prompt Template  

En esta sección se define el **rol del asistente** y la forma en que se construyen los mensajes de entrada y salida. Esto asegura que el sistema tenga un estilo consistente al responder preguntas legales.  

### ¿Qué se hace aquí?  
1. **Importación de ChatPromptTemplate**: se utiliza de `langchain.prompts` para crear plantillas de interacción entre usuario y asistente.  
2. **Definición del rol del asistente**: mediante `system_prompt`, se establece que el modelo actúe como un **asistente legal especializado en leyes de Colombia**, respondiendo de forma clara, breve y en español sencillo.  
3. **Creación de la plantilla de mensajes**: con `ChatPromptTemplate.from_messages`, se combinan las instrucciones del sistema y la pregunta del usuario (`{input}`).  
4. **Integración con el RAG chain**: el prompt se conecta a la cadena de recuperación y generación (`qa_chain`), aplicando el contexto antes de responder.  
5. **Confirmación por consola**: se imprime un mensaje indicando que el Prompt Template fue configurado correctamente.  



In [5]:
# MENSAJES Y PROMPT TEMPLATE

from langchain.prompts import ChatPromptTemplate

# Definimos el prompt para que el asistente tenga un rol
system_prompt = """
Eres un asistente legal especializado en leyes de Colombia.
Responde siempre de forma clara, breve y en español sencillo.
Cuando cites un artículo o norma, menciona el número y de qué documento viene.
Si no sabes la respuesta con certeza, dilo claramente.
"""

# Plantilla para combinar contexto + pregunta
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Conectar el prompt con el RAG chain que ya creaste
qa_chain_with_prompt = prompt | qa_chain

print("✅ Prompt Template configurado. El asistente ahora tiene rol y estilo definido.")


✅ Prompt Template configurado. El asistente ahora tiene rol y estilo definido.


# Paso 1 - Output Parser (Pydantic)  

En esta sección se configura un **parser estructurado con Pydantic** que define cómo deben formatearse las respuestas generadas por el modelo. Esto asegura que las salidas sigan un formato JSON válido y consistente.  

### ¿Qué se hace aquí?  
1. **Importación de librerías**: se carga `BaseModel` de Pydantic y `PydanticOutputParser` desde diferentes ubicaciones de LangChain (para asegurar compatibilidad según la versión).  
2. **Definición del modelo de salida**: se crea la clase `StructuredAnswer` con los campos `answer` (respuesta principal), `sources` (lista de fuentes) y `follow_up_questions` (preguntas relacionadas).  
3. **Inicialización del parser**: se instancia `PydanticOutputParser` indicando que debe usar el modelo `StructuredAnswer` como esquema de validación.  
4. **Obtención de instrucciones de formato**: con `parser.get_format_instructions()` se generan las reglas que el modelo debe seguir para devolver la respuesta en JSON válido.  
5. **Construcción de un PromptTemplate con formato**: se crea un `PromptTemplate` que combina las instrucciones de formato, el contexto y la pregunta del usuario. De este modo, el modelo sabrá exactamente cómo estructurar la respuesta.  
6. **Mensajes de confirmación**: se imprime en consola que el parser fue inicializado y está listo para usarse.  


In [6]:
# CELDA: 1 - Output parser (Pydantic)
from pydantic import BaseModel
try:
    from langchain_core.output_parsers.pydantic import PydanticOutputParser
except Exception:
    try:
        from langchain.output_parsers import PydanticOutputParser
    except Exception:
        from langchain_core.output_parsers.pydantic import PydanticOutputParser

class StructuredAnswer(BaseModel):
    answer: str
    sources: list[str] = []
    follow_up_questions: list[str] = []

parser = PydanticOutputParser(pydantic_object=StructuredAnswer)

# --- NUEVO: obtener instrucciones de formato ---
format_instructions = parser.get_format_instructions()
print("✅ format_instructions ready.")
print(format_instructions)  # opcional, para que veas cómo se debe devolver el JSON

# PromptTemplate ya con las instrucciones inyectadas
from langchain.prompts import PromptTemplate
prompt_with_format = PromptTemplate(
    template=(
        "Usa únicamente la información del contexto para responder. "
        "Devuelve un JSON válido con el formato indicado.\n\n"
        "{format_instructions}\n\n"
        "Contexto:\n{context}\n\nPregunta:\n{question}"
    ),
    input_variables=["context", "question"],
    partial_variables={"format_instructions": format_instructions}
)

print("✅ Parser inicializado correctamente y listo para usarse.")


✅ format_instructions ready.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"answer": {"title": "Answer", "type": "string"}, "sources": {"default": [], "items": {"type": "string"}, "title": "Sources", "type": "array"}, "follow_up_questions": {"default": [], "items": {"type": "string"}, "title": "Follow Up Questions", "type": "array"}}, "required": ["answer"]}
```
✅ Parser inicializado correctamente y listo para usarse.


# Paso 8 - Memoria Conversacional  

En esta sección se añade una **memoria de conversación** que permite al asistente recordar lo que el usuario ha dicho previamente y responder de forma coherente a lo largo del diálogo.  

### ¿Qué se hace aquí?  
1. **Importación de componentes**: se usan `ConversationBufferMemory` para almacenar el historial y `ConversationChain` para integrar la memoria con el modelo.  
2. **Creación de la memoria**: se instancia `ConversationBufferMemory()`, que guarda todo el historial de la conversación de forma secuencial.  
3. **Integración con el modelo (LLM)**: se conecta el modelo de lenguaje (`llm`) con la memoria mediante `ConversationChain`, logrando que las respuestas tengan contexto.  
4. **Confirmación por consola**: se imprime un mensaje que indica que la memoria conversacional fue inicializada correctamente.  
5. **Prueba de funcionamiento**:  
   - Se envía el mensaje `"Hola, soy Oscar"`.  
   - Luego, el asistente recuerda lo dicho y responde correctamente cuando se le pregunta `"¿Cuál es mi nombre?"`.  


In [7]:
# CELDA: 8 - Memoria conversacional

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Creamos la memoria (guarda TODO el historial de la conversación)
memory = ConversationBufferMemory()

# Conectamos el modelo (llm) con la memoria
conversation = ConversationChain(
    llm=llm,        # usa el llm que ya definiste en las celdas anteriores
    memory=memory
)

print("✅ Memoria conversacional inicializada correctamente.")

# --- PRUEBA ---
print(conversation.predict(input="Hola, soy Oscar"))
print(conversation.predict(input="¿Cuál es mi nombre?"))


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_6540\132101666.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_6540\132101666.py:10: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


✅ Memoria conversacional inicializada correctamente.
Hola Oscar, es un placer conocerte. Me alegra que hayas decidido hablar conmigo hoy. ¿En qué puedo ayudarte? ¿Quieres hablar sobre algo en particular o simplemente charlar un rato? Por cierto, sé que estás en un país de habla hispana, pero no puedo determinar exactamente en qué país o ciudad te encuentras. ¿Podrías decirme un poco más sobre ti y dónde te encuentras?
Disculpa la confusión inicial. Me parece que cometí un error al mencionar tu nombre. Como acabas de presentarte, tu nombre es Oscar. ¿Quieres seguir hablando sobre ti mismo o prefieres cambiar de tema? ¿Te gustaría saber algo sobre la tecnología detrás de mí o sobre algún tema en particular?


# Paso 9 - Crear Agente Híbrido Simplificado (Groq principal + Gemini fallback)  

En esta sección se construye un **agente híbrido** que combina dos modelos de lenguaje (Groq y Gemini). El sistema intenta usar **Groq como modelo principal**, pero si falla, automáticamente activa **Gemini como respaldo**. Además, el agente puede usar herramientas externas como el buscador en PDFs y la carga de páginas web.  

### ¿Qué se hace aquí?  
1. **Importación de librerías**: se utilizan módulos de LangChain, Groq, Gemini y loaders web para construir el agente y sus herramientas.  
2. **Definición de herramientas personalizadas**:  
   - `search_index_tool`: consulta información en los documentos PDF indexados mediante el RAG (`qa_chain`).  
   - `fetch_url_tool`: carga el contenido de una página web dada una URL.  
   Ambas herramientas se agrupan en la lista `tools`, con nombre, descripción y comportamiento configurado.  
3. **Definición de modelos LLM**:  
   - `llm_groq`: modelo principal (`llama-3.1-8b-instant`) con temperatura 0.  
   - `llm_gemini`: modelo de respaldo (`gemini-2.5-flash`) también con temperatura 0.  
4. **Creación de un prompt inicial**: se establece un rol de **asistente universitario** con reglas específicas (usar cada herramienta solo una vez, detenerse tras el “Fin de búsqueda”, evitar repeticiones).  
5. **Función `create_hybrid_agent`**:  
   - Intenta inicializar el agente con **Groq**.  
   - Si Groq falla, captura la excepción y activa **Gemini como fallback**.  
   - En ambos casos, el LLM elegido queda definido como variable global (`llm`).  
   - Se imprimen mensajes de estado para confirmar el modelo en uso.  
6. **Creación del agente híbrido**: se llama a `create_hybrid_agent()` para instanciar el agente listo para responder preguntas.  
7. **Ejemplo de prueba**: el agente recibe la pregunta `"¿Qué información hay en el PDF sobre el proceso de inscripción?"`, consulta las fuentes disponibles y devuelve la **respuesta final** al usuario.  



In [8]:
# ============================================
# PASO 9 - CREAR AGENTE HÍBRIDO SIMPLIFICADO
# (Groq principal + Gemini fallback)
# ============================================

from langchain.agents import Tool, initialize_agent, AgentType
from langchain_community.document_loaders import WebBaseLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage

# -------- 1. Definir herramientas --------

def search_index_tool(query: str):
    """Usa el RAG (qa_chain) para buscar información en los documentos indexados"""
    result = qa_chain.run(query)
    return f"Respuesta encontrada en el PDF:\n\n{result}\n\n(Fin de búsqueda)"

def fetch_url_tool(url: str):
    """Carga el contenido de una página web desde su URL"""
    try:
        loader = WebBaseLoader(url)
        docs = loader.load()
        return docs[0].page_content[:1000] + "\n\n(Fin de búsqueda)"
    except Exception as e:
        return f"Error al cargar la URL: {e}"

tools = [
    Tool(
        name="SearchIndex",
        func=search_index_tool,
        description="Usa esto para buscar en los documentos PDF o indexados.",
        return_direct=True
    ),
    Tool(
        name="FetchURL",
        func=fetch_url_tool,
        description="Usa esto para leer contenido de una página web.",
        return_direct=True
    )
]

# -------- 2. Definir modelos LLM --------
llm_groq = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# -------- 3. Prompt inicial --------
system_prompt = SystemMessage(content="""
Eres un asistente universitario que responde preguntas de estudiantes.

Reglas:
- Usa SearchIndex o FetchURL solo una vez si es necesario.
- Si una herramienta devuelve "Fin de búsqueda", da la respuesta final y detente.
- No repitas la misma búsqueda.
""")

# -------- 4. Función para crear agente con fallback --------
def create_hybrid_agent():
    global llm  # 👈 hacemos que quede definido globalmente
    try:
        print("🟢 Inicializando agente con Groq...")
        agent = initialize_agent(
            tools=tools,
            llm=llm_groq,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=4,
            agent_kwargs={"extra_prompt_messages": [system_prompt]}
        )
        _ = llm_groq.invoke("Test rápido")
        llm = llm_groq   # 👈 definimos llm global
        print("✅ Agente con Groq creado y funcionando.")
        return agent
    except Exception as e:
        print(f"⚠️ Groq falló: {e}")
        print("🔵 Usando Gemini como fallback...")
        agent = initialize_agent(
            tools=tools,
            llm=llm_gemini,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=4,
            agent_kwargs={"extra_prompt_messages": [system_prompt]}
        )
        llm = llm_gemini  # 👈 definimos llm global
        print("✅ Agente con Gemini creado y funcionando.")
        return agent

# -------- 5. Crear el agente híbrido --------
agent = create_hybrid_agent()

# -------- 6. Ejemplo de prueba --------
resp = agent.invoke("¿Qué información hay en el PDF sobre el proceso de inscripción?")
print("\n🤖 Respuesta final del agente:\n", resp["output"])


🟢 Inicializando agente con Groq...


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_6540\3940377589.py:62: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


✅ Agente con Groq creado y funcionando.


> Entering new AgentExecutor chain...
Thought: Necesito buscar información sobre el proceso de inscripción en el PDF.
Action: SearchIndex
Action Input: "proceso de inscripción en el PDF"

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_6540\3940377589.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(query)



Observation: Respuesta encontrada en el PDF:

El proceso de inscripción se describe en el siguiente paso a paso:

1. **Diligenciar el formulario de inscripción**: Puedes hacerlo en línea o en medio físico.
2. **Presentar el Examen de Estado o su equivalente**: Si eres extranjero y estás ingresando por primera vez, debes presentar el Examen de Estado o su equivalente, realizado en el lugar donde culminaste tus estudios de educación media.
3. **Anexar documentos adicionales**: Dependiendo de la solicitud que estés realizando, debes anexar los siguientes documentos:
 * **Reingreso, simultaneidad, segunda carrera o transferencia interna**: Anexa una fotocopia ampliada del documento de identidad, si existe cambio del documento.
 * **Transferencia externa**: Anexa la solicitud de homologaciones, certificado original de calificaciones, fotocopia certificada de los contenidos sintéticos y constancia de buena conducta.
 * **Reingreso, transferencias internas y obligatorias**: Anexa paz y salvo

***imports y creación de widgets***

In [9]:
# ===== Celda 1: Imports y definición de widgets =====
import os, tempfile, traceback
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widgets: subida, opciones y entradas
upload = widgets.FileUpload(accept='.pdf', multiple=True, description="Subir PDFs")
chunk_size_dropdown = widgets.Dropdown(options=[500, 800, 1000, 1500], value=1000, description='chunk_size:')
chunk_overlap_text = widgets.IntText(value=200, description='chunk_overlap:')
btn_ingest = widgets.Button(description="Ingestar y Persistir", button_style='success')

mode_dropdown = widgets.Dropdown(
    options=[
        'RetrievalQA (qa_chain)', 
        'Conversational (memory)', 
        'Agent ReAct'
    ],
    value='RetrievalQA (qa_chain)',
    description='Modo:'
)
k_dropdown = widgets.Dropdown(options=[1,2,3,4,5,6], value=3, description='k (recuperar):')

txt_question = widgets.Text(placeholder="Escribe tu pregunta...", description="Pregunta:", layout=widgets.Layout(width="70%"))
btn_ask = widgets.Button(description="Consultar", button_style='info')
btn_run_agent = widgets.Button(description="Ejecutar Agente", button_style='warning')

# Checkbox para aplicar parser estructurado (opcional)
use_parser_chk = widgets.Checkbox(value=False, description="Formatear salida con parser (JSON)")

# Área de salida
out = widgets.Output(layout={'border': '1px solid black'})

# Muestra la UI
top_row = widgets.HBox([upload, chunk_size_dropdown, chunk_overlap_text, btn_ingest])
mid_row = widgets.HBox([mode_dropdown, k_dropdown, use_parser_chk])
ask_row = widgets.HBox([txt_question, btn_ask, btn_run_agent])
display(widgets.VBox([top_row, mid_row, ask_row, out]))


In [10]:
# ===== Celda 2: Funciones utilitarias =====
import io, os, tempfile, traceback
from pathlib import Path

# --- Ingesta de PDFs y creación del vectorstore ---
def ingest_documents_from_file(path, chunk_size=1000, chunk_overlap=200, persist_dir="chroma_db"):
    """Carga PDF -> split -> embeddings -> Chroma (persist)."""
    try:
        from langchain.document_loaders import PyPDFLoader
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        from langchain.embeddings import OpenAIEmbeddings
        from langchain.vectorstores import Chroma
    except Exception as e:
        raise RuntimeError("Faltan paquetes de LangChain o imports: " + str(e))
    
    loader = PyPDFLoader(path)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_documents(docs)
    
    emb = globals().get("embedding_model", None) or OpenAIEmbeddings()
    vectordb = Chroma.from_documents(documents=chunks, embedding_function=emb, persist_directory=persist_dir)
    vectordb.persist()
    
    return {"chunks": len(chunks), "persist_dir": persist_dir}

def load_persisted_vectorstore(persist_dir="chroma_db"):
    """Carga Chroma persistida y devuelve el vectorstore."""
    try:
        from langchain.vectorstores import Chroma
        from langchain.embeddings import OpenAIEmbeddings
    except Exception as e:
        raise RuntimeError("Faltan paquetes de LangChain o imports: " + str(e))
    
    emb = globals().get("embedding_model", None) or OpenAIEmbeddings()
    vectordb = Chroma(persist_directory=persist_dir, embedding_function=emb)
    return vectordb

def make_retriever(k=3, persist_dir="chroma_db"):
    """Devuelve un retriever (intenta usar variable `vectorstore` si existe)."""
    if "vectorstore" in globals():
        vs = globals()["vectorstore"]
    elif Path(persist_dir).exists():
        vs = load_persisted_vectorstore(persist_dir)
    else:
        raise RuntimeError(f"No encuentro vectorstore ni {persist_dir}. Ingresa datos primero.")
    return vs.as_retriever(search_kwargs={"k": k})

# --- Selección del chain o agente según el modo ---
def ensure_chain_for_mode(mode, k=3, memory_obj=None):
    """Crea y devuelve SOLO el objeto chain según el modo seleccionado."""
    try:
        from langchain.chains import RetrievalQA, ConversationalRetrievalChain
        from langchain.memory import ConversationBufferMemory
        from langchain.agents import Tool, initialize_agent, AgentType
    except Exception as e:
        raise RuntimeError("Faltan imports LangChain: " + str(e))
    
    if "llm" not in globals():
        raise RuntimeError("No se ha definido 'llm'. Define Gemini/ChatGoogleGenerativeAI antes.")
    
    retriever = make_retriever(k=k)

    if mode.startswith("RetrievalQA"):
        return RetrievalQA.from_chain_type(
            llm=globals()["llm"],
            retriever=retriever,
            chain_type="stuff",
            return_source_documents=True
        )

    elif mode.startswith("Conversational"):
        memory = memory_obj or ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"   # 👈 clave de salida definida
        )
        return ConversationalRetrievalChain.from_llm(
            llm=globals()["llm"],
            retriever=retriever,
            memory=memory,
            return_source_documents=True,
            output_key="answer"   # 👈 clave de salida consistente
        )

    elif mode.startswith("Agent"):
        # Herramienta de búsqueda sobre el retriever (evitar invoke directo)
        def search_tool(q: str):
            docs = retriever.get_relevant_documents(q)
            return "\n\n".join([d.page_content[:1000] for d in docs])

        tools = [
            Tool(
                name="search_docs",
                func=search_tool,
                description="Busca en la base vectorial y devuelve extractos relevantes."
            )
        ]
        return initialize_agent(
            tools,
            globals()["llm"],
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=False,
            handle_parsing_errors=True  # evita que se quede colgado
        )

    else:
        raise ValueError(f"Modo no reconocido: {mode}")

# --- Handler para ingestar documentos ---
def on_ingest_clicked(b):
    btn_ingest.disabled = True
    with out:
        clear_output()
        print("Iniciando ingesta de documentos PDF...")
        try:
            if not file_upload.value:
                print("⚠️ No hay archivo cargado. Sube un PDF primero.")
                btn_ingest.disabled = False
                return
            
            fname, fdata = next(iter(file_upload.value.items()))
            path = os.path.join(tempfile.gettempdir(), fname)
            with open(path, "wb") as f:
                f.write(fdata["content"])
            
            res = ingest_documents_from_file(path)
            globals()["vectorstore"] = load_persisted_vectorstore(res["persist_dir"])
            print(f"✅ Ingesta completada. {res['chunks']} fragmentos guardados en {res['persist_dir']}")
        except Exception as e:
            print("❌ Error en ingesta:", type(e).__name__, e)
            traceback.print_exc()
        finally:
            btn_ingest.disabled = False

# --- Handler de consulta (usa el modo seleccionado) ---
def on_ask_clicked(b):
    btn_ask.disabled = True
    with out:
        clear_output()
        q = txt_question.value.strip()
        if not q:
            print("Escribe una pregunta primero.")
            btn_ask.disabled = False
            return
        
        mode = mode_dropdown.value
        k = int(k_dropdown.value)
        print(f"Modo: {mode}  ·  k={k}")
        
        try:
            chain_obj = ensure_chain_for_mode(mode, k=k)
            if mode.startswith("RetrievalQA"):
                res = chain_obj({"query": q})
                ans = res.get("result") or ""
                docs = res.get("source_documents") or []
            elif mode.startswith("Conversational"):
                res = chain_obj({"question": q})
                ans = res.get("answer") or ""
                docs = res.get("source_documents") or []
            else:  # Agent
                ans = chain_obj.run(q)
                docs = []
            
            print("=== RESPUESTA ===")
            print(ans)
            if docs:
                print("\n=== FUENTES (top) ===")
                for d in docs[:k]:
                    src = d.metadata.get("source", "sin fuente")
                    excerpt = (d.page_content[:300] + "...") if len(d.page_content) > 300 else d.page_content
                    print("-", src, "|", excerpt)
            
            if use_parser_chk.value:
                print("\nAplicando parser estructurado (puede usar más tokens)...")
                parsed = format_answer_with_parser(q)
                print("Parsed output:", parsed)
        except Exception as e:
            print("Error al consultar:", type(e).__name__, e)
            traceback.print_exc()
        finally:
            btn_ask.disabled = False

# --- Handler para ejecutar el agente por separado ---
def on_run_agent_clicked(b):
    btn_run_agent.disabled = True
    with out:
        clear_output()
        q = txt_question.value.strip()
        if not q:
            print("Escribe una pregunta para el agente.")
            btn_run_agent.disabled = False
            return
        try:
            agent_obj = ensure_chain_for_mode("Agent", k=int(k_dropdown.value))
            print("Ejecutando agente ReAct...")
            out_agent = agent_obj.invoke(q)  # uso invoke en vez de run
            print("Respuesta agente:\n", out_agent)
        except Exception as e:
            print("Error agente:", type(e).__name__, e)
            traceback.print_exc()
        finally:
            btn_run_agent.disabled = False


In [11]:
# ===== Celda 3: Conectar eventos (ejecutar al final) =====
# Conectar botones y enter en la caja de texto (observe para evitar on_submit deprecado)
btn_ingest.on_click(on_ingest_clicked)
btn_ask.on_click(on_ask_clicked)
btn_run_agent.on_click(on_run_agent_clicked)

def on_text_change(change):
    # Se dispara cada vez que cambia el texto; para ejecutar con Enter, lo detectamos
    # Algunos kernels ponen el valor vacío en cada tecla; para evitar llamadas múltiples, ejecuta solo cuando el usuario pulsa Enter:
    # Observación: detectaremos Enter si el último caracter es '\n' (puede variar). Aquí usamos solo botón como método principal.
    pass

# Mejor: permitir Enter simulando click si el usuario presiona Enter (algunos entornos no envían evento distinto).
# Aquí conectamos 'value' observe y solo llamamos si el valor nuevo difiere y el botón no está en proceso.
def on_text_submit(change):
    # No queremos que se llame en cada tecla; hacemos un simple umbral: si termina con '⏎' no fiable.
    # Mantendremos el botón como principal; pero permitimos una llamada si el usuario borra/introduce texto y luego presiona Enter
    # Para mayor compatibilidad, deja el botón como la forma confiable.
    return

txt_question.observe(lambda ch: None, names='value')  # placeholder, no hacemos nada para evitar duplicar llamadas

# Mensaje de estado rápido
with out:
    clear_output()
    print("Handlers conectados. Verifica que 'llm' esté definido y que 'vectorstore' exista (o usa el botón Ingestar).")
    if "llm" not in globals():
        print("⚠️ Aviso: no detecté 'llm' en el notebook. Define tu LLM (ej. ChatGoogleGenerativeAI) antes de preguntar.")
    if "vectorstore" in globals():
        print("Vectorstore detectado en memoria. Puedes consultar directamente.")
    elif os.path.exists("chroma_db"):
        print("Se detecta carpeta 'chroma_db' en disco; se cargará automáticamente al consultar.")
    else:
        print("No se encontró vectorstore persistido. Usa 'Ingestar y Persistir' para crear 'chroma_db'.")
